<a href="https://colab.research.google.com/github/lena110200/Class2022-spring/blob/master/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

### [Image classification](https://huggingface.co/tasks/image-classification)

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use
#웹페이지에 image classification정의 나옴 

In [ ]:
!pip install transformers # transformer 먼저 install시켜줘야 

In [ ]:
#how to use copy / image를 classify함 
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests 

url = 'http://images.cocodataset.org/val2017/000000039769.jpg' #url안에 그림 
image = Image.open(requests.get(url, stream=True).raw) #이미지를 open해서. image안에 숫자형태로
#jpg에 있는 정보가 숫자 형태로 들어옴 
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')#pretrained : 행렬 
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')#extract된 feature정보를 classify

inputs = feature_extractor(images=image, return_tensors="pt") #이미지 정보로부터 feature정보를 뽑아냄
#original image보다 input이 적음. 
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes - 천개의 클래스에 대해 확률정보 (100개의 숫자)
#천개 숫자 다더하면 1 
predicted_class_idx = logits.argmax(-1).item() #천개의 숫자중 확률이 젤 큰거 찾아라 - 한개 
print("Predicted class:", model.config.id2label[predicted_class_idx])

Demo in Gradio

In [ ]:
def func (image): #함수 
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#gradio : 위의 함수의 입출력의 유형을 정의 해줘야. 이미지를 넣고 출력으로 text. 
#이미지 파일 두개가 있다는걸 assume 

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased') #모델을 들고와서 이름을 unmask로 
unmasker("Hello I'm a [MASK] model.") 
#dict가 list로 묶여서 나옴 

Demo in Gradio

In [ ]:
import pandas as pd
def func (text): #text를 입력으로 받아서 
  unmasker = pipeline('fill-mask', model='bert-base-uncased') #모델 불러와서 
  result = unmasker(text) #
  df = pd.DataFrame(result) # list를 dataframe화 
  return df

In [ ]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()
#위의 잇는 예문들을 list에 넣어  

### [Token classification](https://huggingface.co/tasks/token-classification)

In [ ]:
#natural lg understanding - natural lg가 주어져있다면 그게 자유도가 높은데 필요한 정보만 뽑아내야.
#NER : 문장이 주어져있을때 어떤 명사는 더 정보적으로 중요. 더 관심을가져야. 그런것들을 인식. 
# 위치 정보, 단체정보, 사람.. 

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#각 단어들을 classify 
nlp = pipeline("ner", model=model, tokenizer=tokenizer) #nlp가 모델 
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example) #위의 있는 문장을 가지고 모델을 돌림 
print(ner_results) # 각 토큰에 대해 설명 

Demo in Gradio

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result) #최종 result를 df로 받아 
  return df

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
# 두 주어진 문장이 있을때 얼만큼 유사한가 / 모든 모델들은 input을 바로 받아들일 수 없음 
#raw data를 모델이 그대로 받아들이기 힘듬. feature형태로 . embedding형태로 바꿔줘 

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings) 
#embiddings : list 2개 / 숫자 총 384 / 384개의 숫자가 [...] [...] : []하나가 embedding 
#list안에 있는게 feature vector 숫자들 data -> 이걸 가지고 similarity계산 


In [ ]:
len(embeddings[1])

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1]) # 첫번째 문장에 대한 384개의 숫자, 두번째 문장 =
#두백타가 이루는 값의 코사인 계산 
cosine_scores

Demo in Gradio

In [ ]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [ ]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()